##### Installing the required libraries

In [ ]:
#%pip install weaviate-client llama-index-vector-stores-weaviate

##### Loading dotenv file

In [1]:
from dotenv import load_dotenv
import os

# for loading GROQ_API_KEY and WEAVIATE_URL + WEAVIATE_API_KEY
load_dotenv()

True

##### Importing libraries

In [2]:
from llama_index.core import (
    Settings, # setting the configuration of LLM to be used
    VectorStoreIndex, # default vector store indexing used in llamaindex
    SimpleDirectoryReader, # reading directory containing data files
    StorageContext
)
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding # using HuggingFace embeddings
from llama_index.llms.groq import Groq # using Groq LLM 
import weaviate

##### Initializing APIs and connecting to Weaviate cluster

In [10]:
GROQ_API_KEY = os.environ["GROQ_API_KEY"]
URL = os.environ["WEAVIATE_URL"]
APIKEY = os.environ["WEAVIATE_API_KEY"]

client = weaviate.connect_to_wcs(
    cluster_url = URL, 
    auth_credentials = weaviate.auth.AuthApiKey(APIKEY)
)

##### Setting LLM configuration for LlamaIndex

In [ ]:
llm = Groq(model = "llama3-8b-8192")
embed_model = HuggingFaceEmbedding(model_name = "sentence-transformers/all-mpnet-base-v2")

Settings.llm = llm
Settings.embed_model = embed_model

##### Loading data from directory using LlamaIndex

In [12]:
documents = SimpleDirectoryReader("data_files").load_data()

##### Loading data in Weaviate

In [13]:
vector_store = WeaviateVectorStore(
    weaviate_client = client, index_name = "Index101"
    )
storage_context = StorageContext.from_defaults(vector_store = vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context = storage_context
)

##### Executing query using LlamaIndex and Weaviate vector db

In [14]:
query_engine = index.as_query_engine()
response = query_engine.query("Who won the 2024 presidential elections in USA?")
print(response)

Donald Trump and JD Vance.
